In [1]:
import pandas as pd
import requests
import json
import csv
import datetime
import glob

In [2]:
directory = r'C:\Users\Hannah\Documents\Insulin Advocacy\Price Graph'
insulin_price_data = pd.read_csv(directory + r'\humalog_prices.csv')

# Load in Price Data (Truven)

In [3]:
insulin_price_data['Price'] = insulin_price_data['Price'].apply(lambda x: x.replace('$', '').replace(',', '')
                                if isinstance(x, str) else x).astype(float)

insulin_price_data['Date'] = pd.to_datetime(insulin_price_data['Date'],format='%m/%d/%Y').dt.date
insulin_price_data = insulin_price_data.dropna(how='all')

In [4]:
insulin_price_data.tail()

,Date,Price
31,2014-11-25,202.6
32,2015-05-29,222.7
33,2015-12-01,237.0
34,2016-07-13,254.8
35,2017-05-02,274.7


In [5]:
price_data_as_list = insulin_price_data.values.tolist()

## Normalize to get price at year end

In [6]:
data_holder_1 = [] 
for i, stamp in enumerate(price_data_as_list):
    #print(stamp)
    price_change_per_day = -1
    if i == len(price_data_as_list) - 1:
        pass
    else:
        date_diff = (price_data_as_list[i+1][0] - stamp[0]).days
        
        price_change_per_day = (price_data_as_list[i+1][1] - stamp[1])/date_diff
    data_holder_1.append(stamp + [price_change_per_day])

In [7]:
year_end_dates = ['{}-12-31'.format(x) for x in range(1996, 2017)]

In [8]:
data_holder_2 = []
for x in year_end_dates:
    year_end_as_date = datetime.datetime.strptime(x , '%Y-%m-%d')
    
    eyes = []
    for i, stamp in enumerate(data_holder_1):
        stamp_date = datetime.datetime.combine(stamp[0], datetime.time(0, 0))
        if year_end_as_date > stamp_date:
            eyes.append(i)
    max_i = max(eyes)
    
    compare_date = datetime.datetime.combine(data_holder_1[max_i][0], datetime.time(0, 0))
    date_difference = (year_end_as_date - compare_date).days
    factor = date_difference * data_holder_1[max_i][2]
    price_at_year_end = data_holder_1[max_i][1] + factor
    
    print(year_end_as_date,data_holder_1[max_i][0],data_holder_1[max_i][1],data_holder_1[max_i][2], date_difference, factor, price_at_year_end)
    
    data_holder_2.append(['N/A', 'Insulin (Humalog)', year_end_as_date, price_at_year_end])

1996-12-31 00:00:00 1996-07-24 20.82 0.002556390977443608 160 0.40902255639097723 21.229022556390976
1997-12-31 00:00:00 1997-08-27 21.84 0.004540059347181012 126 0.5720474777448076 22.412047477744807
1998-12-31 00:00:00 1998-07-30 23.37 0.013680781758957653 154 2.1068403908794786 25.47684039087948
1999-12-31 00:00:00 1999-06-02 27.57 0.005594202898550724 212 1.1859710144927536 28.755971014492754
2000-12-31 00:00:00 2000-11-21 34.81 0.030683229813664583 40 1.2273291925465832 36.037329192546586
2001-12-31 00:00:00 2001-05-01 39.75 0.007653846153846161 244 1.8675384615384634 41.617538461538466
2002-12-31 00:00:00 2002-12-19 49.14 0.017065637065637074 12 0.20478764478764488 49.344787644787644
2003-12-31 00:00:00 2003-09-04 53.56 0.012422680412371117 118 1.4658762886597918 55.025876288659795
2004-12-31 00:00:00 2004-03-16 55.97 0.01341296928327645 290 3.88976109215017 59.859761092150165
2005-12-31 00:00:00 2005-07-27 64.69 0.01050925925925924 157 1.6499537037037009 66.3399537037037
2006-12

# Load in Inflation Data (BLS)

In [9]:
inflation_dict = {'CUUR0000SEMD': 'Hospital Services', 'CUUR0000SEMF01': 'Prescription Drugs', 'CUUR0000SAM2':'Medical Care Services', 'CUUR0000SEEB01': 'College Tuition',
                 'CUUR0000SEEB03': 'Childcare', 'CUUR0000SAH1': 'Housing (Shelter)', 'CUUR0000SAH2': 'Utilities', 'CUUR0000SAF': 'Food and Beverage', 'CUUR0000SEED03': 'Cellphone Service', 'CUUR0000SA0': 'All Items'}

In [10]:
series_list = [k  for k in inflation_dict]

## Download data from BLS in chunks

In [11]:
'''
start_ = 2016
end_ = 2017
#header for the API
headers = {'Content-type': 'application/json'}
# list of data sets we want to get, with dates
data = json.dumps({"seriesid": series_list, "startyear":"{}".format(start_), "endyear":"{}".format(end_)})

# go and get the data and load it as a json file
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
'''

'\nstart_ = 2016\nend_ = 2017\n#header for the API\nheaders = {\'Content-type\': \'application/json\'}\n# list of data sets we want to get, with dates\ndata = json.dumps({"seriesid": series_list, "startyear":"{}".format(start_), "endyear":"{}".format(end_)})\n\n# go and get the data and load it as a json file\np = requests.post(\'https://api.bls.gov/publicAPI/v2/timeseries/data/\', data=data, headers=headers)\njson_data = json.loads(p.text)\n'

In [12]:
'''
fields=["series_id","year","period","value","footnotes"]

for series in json_data['Results']['series']:
    
    seriesID = series['seriesID']
    outfile = '{}/data/{}_{}.csv'.format(directory, seriesID, end_)
    
    with open(outfile, 'w') as iFile:
        writer = csv.writer(iFile)
        writer.writerow(fields)
        
        for item in series['data']:
            year = item['year']
            period = item['period']
            value = item['value']
            footnotes=""
            for footnote in item['footnotes']:
                if footnote:
                    footnotes = footnotes + footnote['text'] + ','

            if 'M01' <= period <= 'M12':
                writer.writerow([seriesID,year,period,value,footnotes[0:-1]])
                
    iFile.close()  # good idea to close if you're done with it
    '''

'\nfields=["series_id","year","period","value","footnotes"]\n\nfor series in json_data[\'Results\'][\'series\']:\n    \n    seriesID = series[\'seriesID\']\n    outfile = \'{}/data/{}_{}.csv\'.format(directory, seriesID, end_)\n    \n    with open(outfile, \'w\') as iFile:\n        writer = csv.writer(iFile)\n        writer.writerow(fields)\n        \n        for item in series[\'data\']:\n            year = item[\'year\']\n            period = item[\'period\']\n            value = item[\'value\']\n            footnotes=""\n            for footnote in item[\'footnotes\']:\n                if footnote:\n                    footnotes = footnotes + footnote[\'text\'] + \',\'\n\n            if \'M01\' <= period <= \'M12\':\n                writer.writerow([seriesID,year,period,value,footnotes[0:-1]])\n                \n    iFile.close()  # good idea to close if you\'re done with it\n    '

# Final Data Merging

In [13]:
all_files = glob.glob('{}/data'.format(directory) + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [14]:
frame.head()

,series_id,year,period,value,footnotes
0,CUUR0000SA0,2005,M12,196.8,NaN
1,CUUR0000SA0,2005,M11,197.6,NaN
2,CUUR0000SA0,2005,M10,199.2,NaN
3,CUUR0000SA0,2005,M09,198.8,NaN
4,CUUR0000SA0,2005,M08,196.4,NaN


In [15]:
frame['Category']= ''
frame = frame[(frame['period'] == 'M12') & (frame['year'] < 2017) & (frame['series_id'] != 'CUUR0000SEED03')]
frame['year'] = frame['year'].astype(str)
frame['year_end_date'] = frame['year'] + '-12-31'
frame['year_end_date'] = pd.to_datetime(frame['year_end_date'],format='%Y-%m-%d').dt.date

In [16]:
keep = ['series_id', 'Category', 'year_end_date','value' ]

In [17]:
frame = frame[keep]

In [18]:
frame.head()

,series_id,Category,year_end_date,value
0,CUUR0000SA0,,2005-12-31,196.8
12,CUUR0000SA0,,2004-12-31,190.3
24,CUUR0000SA0,,2003-12-31,184.3
36,CUUR0000SA0,,2002-12-31,180.9
48,CUUR0000SA0,,2001-12-31,176.7


In [19]:
insulin_frame = pd.DataFrame(data_holder_2, columns=['series_id', 'Category', 'year_end_date','value'])
insulin_frame['year_end_date'] = insulin_frame['year_end_date'].apply(lambda x: x.date())

In [20]:
combined_frame = pd.concat([insulin_frame, frame])


In [21]:
combined_frame.loc[(combined_frame['series_id'] == 'CUUR0000SEMD'),'Category'] = 'Hospital Services'
combined_frame.loc[(combined_frame['series_id'] == 'CUUR0000SEMF01'),'Category'] = 'Prescription Drugs'
combined_frame.loc[(combined_frame['series_id'] == 'CUUR0000SAM2'),'Category'] = 'Medical Care Services'
combined_frame.loc[(combined_frame['series_id'] == 'CUUR0000SEEB01'),'Category'] = 'College Tuition'
combined_frame.loc[(combined_frame['series_id'] == 'CUUR0000SEEB03'),'Category'] = 'Childcare'
combined_frame.loc[(combined_frame['series_id'] == 'CUUR0000SAH1'),'Category'] = 'Housing (Shelter)'
combined_frame.loc[(combined_frame['series_id'] == 'CUUR0000SAH2'),'Category'] = 'Utilities'
combined_frame.loc[(combined_frame['series_id'] == 'CUUR0000SAF'),'Category'] = 'Food and Beverage'
combined_frame.loc[(combined_frame['series_id'] == 'CUUR0000SA0'),'Category'] = 'All Items'

In [22]:
combined_frame = combined_frame.sort_values(['series_id','Category', 'year_end_date'], ascending=[0,1, 1])
combined_frame_list = combined_frame.values.tolist()

In [23]:
combined_frame.tail(30)

,series_id,Category,year_end_date,value
468,CUUR0000SAF,Food and Beverage,2008-12-31,218.839
456,CUUR0000SAF,Food and Beverage,2009-12-31,218.049
444,CUUR0000SAF,Food and Beverage,2010-12-31,221.278
432,CUUR0000SAF,Food and Beverage,2011-12-31,231.130
420,CUUR0000SAF,Food and Beverage,2012-12-31,235.230
408,CUUR0000SAF,Food and Beverage,2013-12-31,237.820
396,CUUR0000SAF,Food and Beverage,2014-12-31,245.585
384,CUUR0000SAF,Food and Beverage,2015-12-31,247.468
516,CUUR0000SAF,Food and Beverage,2016-12-31,247.134
108,CUUR0000SA0,All Items,1996-12-31,158.600


In [24]:
final_data_holder = []
start_value = -1
for i, entry in enumerate(combined_frame_list):
    if entry[2] == datetime.date(1996, 12, 31):
        start_value = entry[3]
        per_change = 0
        
    elif entry[2] == datetime.date(1997, 12, 31) and entry[1] == 'Cellphone Service':
        start_value = entry[3]
        per_change = 0
    else:
        per_change = ((entry[3] - start_value)/start_value)
    print(entry[1], entry[2], start_value, entry[3], per_change)
    final_data_holder.append([entry[0], entry[1], entry[2], entry[3], start_value,  per_change])

Insulin (Humalog) 1996-12-31 21.229022556390976 21.229022556390976 0
Insulin (Humalog) 1997-12-31 21.229022556390976 22.412047477744807 0.05572677301610769
Insulin (Humalog) 1998-12-31 21.229022556390976 25.47684039087948 0.20009483824349242
Insulin (Humalog) 1999-12-31 21.229022556390976 28.755971014492754 0.35455935091254576
Insulin (Humalog) 2000-12-31 21.229022556390976 36.037329192546586 0.6975500919470069
Insulin (Humalog) 2001-12-31 21.229022556390976 41.617538461538466 0.9604076613037252
Insulin (Humalog) 2002-12-31 21.229022556390976 49.344787644787644 1.3244022429064894
Insulin (Humalog) 2003-12-31 21.229022556390976 55.025876288659795 1.5920117679697083
Insulin (Humalog) 2004-12-31 21.229022556390976 59.859761092150165 1.8197134810678999
Insulin (Humalog) 2005-12-31 21.229022556390976 66.3399537037037 2.1249650579758854
Insulin (Humalog) 2006-12-31 21.229022556390976 71.71280442804428 2.3780549357631737
Insulin (Humalog) 2007-12-31 21.229022556390976 80.14695945945945 2.7753

In [25]:
final_df = pd.DataFrame(final_data_holder, columns=['Code','Category', 'Year End', 'Base Price', 'Price at Year End', 'Price Change (Percentage)'])

In [26]:
cross_tab_df = final_df[['Category', 'Year End', 'Price Change (Percentage)'
]]

In [27]:
cross_tab = pd.crosstab(final_df['Year End'], final_df['Category'],values=final_df['Price Change (Percentage)'], aggfunc='mean').reset_index()

In [28]:
cross_tab.columns.tolist()

['Year End',
 'All Items',
 'Childcare',
 'College Tuition',
 'Food and Beverage',
 'Hospital Services',
 'Housing (Shelter)',
 'Insulin (Humalog)',
 'Medical Care Services',
 'Prescription Drugs',
 'Utilities']

# Plotly

In [32]:
# imports
import plotly.express as px 
import pandas as pd 

# data
fig = px.line(cross_tab, 
              x='Year End', 
              y=[ 'All Items','Childcare','College Tuition','Food and Beverage','Hospital Services','Housing (Shelter)', 'Insulin (Humalog)','Medical Care Services','Prescription Drugs','Utilities'],
             )


fig.update_layout(
    title="Price of Insulin vs. the Price of Other Goods",
    xaxis_title="Year End",
    yaxis_title="Percentage Change Since 1996",
    legend_title="Goods",
    font=dict(
        family="Helvetica",
        size=16,
        color="#2F284F"
    )
)

'''
fig['data'][0]['line']['color']="#F36235" #ALL ITEMS
fig['data'][1]['line']['color']="#2F284F" #CHILDCARE
fig['data'][2]['line']['color']="#1A9A8C" #COLLEGE TUITION
fig['data'][3]['line']['color']="#E7D54C" #FOOD
fig['data'][4]['line']['color']="#F36235" #HOSPITAL
fig['data'][5]['line']['color']="#2F284F" #HOUSING
fig['data'][6]['line']['color']="#E71D36" #INSULIN
fig['data'][7]['line']['color']="#1A9A8C" #MEDICAL CARE
fig['data'][8]['line']['color']="#E7D54C" #PRESCRIPTION
fig['data'][9]['line']['color']="#F36235" #UTILITIES
'''

fig.update_yaxes(nticks=20)
fig.update_xaxes(nticks=20)
fig.update_traces(mode="markers+lines")
fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=12,
        font_family="Helvetica"
    )
)



fig.update_yaxes(tickformat="%")

# Show plot 
fig.show()

In [33]:
import plotly.io as pio
pio.write_html(fig, file='index.html', auto_open=True)